In [1]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate, integrate
import theano.tensor as tt
from theano.compile.ops import as_op
from scipy.interpolate import griddata
import theano
import astropy.units as u
from tqdm import tqdm_notebook

import numpy as np
import pandas as pd
#import splat
#import popsims
from astropy.coordinates import SkyCoord, Galactic
#from popsims.galaxy import Pointing, volume_calc, create_pop
import matplotlib.pyplot as plt
#import seaborn as sns
import scipy
#sns.set_style("darkgrid", {"axes.facecolor": ".95"})

#plt.style.use('fivethirtyeight')
#plt.style.use('dark_background')

from  matplotlib.colors import Normalize
import astropy.units as u
import popsims
import matplotlib
from tqdm import tqdm
from tqdm import tqdm
import matplotlib as mpl 
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] =20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['legend.fontsize'] = 20
mpl.rcParams['figure.titlesize'] = 20

from popsims.simulator import Population,simulate_population
from popsims.galaxy import Disk, Halo
from popsims.relations import polynomial_relation
from popsims.backgrounds import simulate_lsst_background_stars
import arviz as az
#import jax

In [2]:
#PUPOSE CHANGE THE IMF POWER-LAW, COMPUTE FINAL OBSERVED DENSITIES
#tutorial for general simulator object 
#https://docs.pymc.io/en/v3/pymc-examples/examples/samplers/SMC-ABC_Lotka-Volterra_example.html 

In [3]:
ra= 2*np.pi*np.random.uniform(0, 1, 10)
dec= np.arccos(2*np.random.uniform(0, 1, 10)-1)-np.pi/2
footprint=SkyCoord(ra=ra*u.radian, dec=dec*u.radian)


In [4]:
grid= np.arange(15, 42)
data= np.random.normal(loc=100, scale=5, size=len(grid))

data

array([ 97.75347178, 102.57773264,  98.87232967, 112.65734804,
       108.30571508,  93.13681422,  95.65592308,  97.58815354,
       102.38132008, 104.56065406,  93.39564019,  97.67763858,
       100.11909421,  96.1983718 ,  98.24699586,  99.08995859,
        99.23355271,  99.23391999,  96.0519251 , 100.19717523,
        99.15266507,  97.55559375,  97.12228215,  98.81554717,
        92.22268167, 102.5629099 ,  97.97527747])

In [5]:
#@as_op(itypes=[tt.dscalar], otypes=tt.dvector)
def simulate_counts(a, b):
    disk= Disk(H=a, L=b)
    samples_tot= simulate_population(disk, footprint, 0.1, 3000, poptype='dwarfs', mag_keys=['LSST_Z'], \
                             galtype='thin_disk', nsample=1e4)
    
    samples_tot['spt']= samples_tot['spt'].apply(np.round)
    counts=samples_tot.groupby('spt').apply(lambda x: len(x))
    
    counts=samples_tot.groupby('spt').apply(lambda x: len(x))
    #
    res= np.array([counts[x] if x in counts.index else 0.0 for x in grid ]).reshape(data.shape)
    return res

def normal_sim(a, b):
    return np.random.normal(a, b, 1000)

In [6]:
 simulate_counts(100, 100).shape, data.shape

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

((27,), (27,))

In [ ]:
with pm.Model() as model_lv:
    a = pm.Uniform("a", 100, 1000)
    b = pm.Uniform("b", 100, 1000)

    sim = pm.Simulator("sim", simulate_counts, params=(a, b), epsilon=10, observed=data)
    trace_lv = pm.sample_smc(kernel="ABC", parallel=False, draws=10)
    idata_lv = az.from_pymc3(trace_lv)

Initializing SMC sampler...
Sampling 4 chains in 4 jobs
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dty

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

...

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/users/caganze/research/popsims/popsims/tools.py:146: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)
/users/caganze/research/popsims/popsims/tools.py:150: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)
/Users/caganze/opt/anaconda3/envs/pymc3_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|███████████████████████████

In [ ]:
az.plot_ppc(idata)

In [ ]:
 #with pm.Model() as model_lv:
 #   pm.Uniform("a", 100, 1000)
 #   res=pm.sample()